# RequestGeneratorV3

In [1]:
from generate_requests_v3 import RequestGeneratorV3

# Generate Input Files

## General Parameters

In [2]:
num_repeats = 10

## Scaling Number of Requests

In [3]:
for num_requests in [100, 500, 1000, 5000]:
    for i in range(num_repeats):
        rg = RequestGeneratorV3(i)
        rg.generate_single_request_injection(0, num_requests)
        filename = "performanceTest1_{}Requests_{}.json".format(num_requests, i)
        rg.save_to_file("data", "performance_input", filename)

Saved to file at 'data\performance_input\performanceTest1_100Requests_0.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_1.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_2.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_3.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_4.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_5.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_6.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_7.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_8.json'
Saved to file at 'data\performance_input\performanceTest1_100Requests_9.json'
Saved to file at 'data\performance_input\performanceTest1_500Requests_0.json'
Saved to file at 'data\performance_input\performanceTest1_500Requests_1.json'
Saved to file at 'data\performance_input\performanceTest1_500Req

## Scaling Number of Telescopes

In [5]:
for num_telescopes in [1, 2, 3, 5, 10, 15, 20]:
    # for num_requests in [500, 1000, 10000]:
# for num_telescopes in [1, 3, 5]:
    for num_requests in [500, 1000, 5000]:
        for i in range(num_repeats):
            rg = RequestGeneratorV3(i, num_telescopes=num_telescopes)
            rg.generate_single_request_injection(0, num_requests)
            filename = "performanceTest2_{}Telescopes_{}Requests_{}.json".format(num_telescopes, num_requests, i)
            rg.save_to_file("data", "performance_input", filename)

Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_0.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_1.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_2.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_3.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_4.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_5.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_6.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_7.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_8.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_500Requests_9.json'
Saved to file at 'data\performance_input\performanceTest2_1Telescopes_1000Requests_0.json'
Saved to 

## Scaling Slice Size

In [6]:
for slice_size in [600, 300, 240, 180, 120, 60, 30]:
# for slice_size in [600, 300, 180]:
    for num_requests in [1000, 5000]:
        for i in range(num_repeats):
            rg = RequestGeneratorV3(i, slice_size=slice_size)
            rg.generate_single_request_injection(0, num_requests)
            filename = "performanceTest3_{}SliceSize_{}Requests_{}.json".format(slice_size, num_requests, i)
            rg.save_to_file("data", "performance_input", filename)

Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_0.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_1.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_2.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_3.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_4.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_5.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_6.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_7.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_8.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_1000Requests_9.json'
Saved to file at 'data\performance_input\performanceTest3_600SliceSize_5000Reque

## Scaling Number of Networks

In [7]:
num_telescopes = 8
num_requests = 1000
for num_networks in [1, 2, 4, 8]:
    for i in range(num_repeats):
        rg = RequestGeneratorV3(i, num_telescopes=num_telescopes, num_networks=num_networks)
        for network in range(num_networks):
            rg.generate_single_request_injection(0, num_requests, network=network)
        filename = "performanceTest5_{}Networks_{}Telescopes_{}RequestsPerNetwork_{}.json".format(num_networks, num_telescopes, num_requests, i)
        rg.save_to_file("data", "performance_input", filename)

Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_0.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_1.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_2.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_3.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_4.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_5.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_6.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_7.json'
Saved to file at 'data\performance_input\performanceTest5_1Networks_8Telescopes_1000RequestsPerNetwork_8.json'
S

## Scaling Number of Networks (with scaling request number)

In [8]:
num_telescopes = 8
gross_requests = 4000
for num_networks in [1, 2, 4, 8]:
    requests_per_network = int(gross_requests / num_networks)
    for i in range(num_repeats):
        rg = RequestGeneratorV3(i, num_telescopes=num_telescopes, num_networks=num_networks)
        for network in range(num_networks):
            rg.generate_single_request_injection(0, requests_per_network, network=network)
        filename = "performanceTest6_{}Networks_{}Telescopes_{}GrossRequests_{}.json".format(num_networks, num_telescopes, gross_requests, i)
        rg.save_to_file("data", "performance_input", filename)

Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_0.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_1.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_2.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_3.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_4.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_5.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_6.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_7.json'
Saved to file at 'data\performance_input\performanceTest6_1Networks_8Telescopes_4000GrossRequests_8.json'
Saved to file at 'data\performance_input\perfo

## Scaling Scheduling Horizon

In [9]:
for horizon_days in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] :
    for num_requests in [1000]:
        for i in range(num_repeats):
            rg = RequestGeneratorV3(i, horizon_days=horizon_days)
            rg.generate_single_request_injection(0, num_requests)
            filename = "performanceTest7_{}HorizonDays_{}Requests_{}.json".format(horizon_days, num_requests, i)
            rg.save_to_file("data", "performance_input", filename)

Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_0.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_1.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_2.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_3.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_4.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_5.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_6.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_7.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_8.json'
Saved to file at 'data\performance_input\performanceTest7_1HorizonDays_1000Requests_9.json'
Saved to file at 'data\performance_input\performanceTest7_2HorizonDays_1000Reque

# Variable Window Test

Conditions for the Variable Window Test (and for the Priority Banding Test):
* Preload majority of the requests at the beginning, in one injection
* Load a small amount of requests in on a regular basis, to simulate a dynamic and overloaded scheduler
* Make sure that the requests are extending past the Simulator Horizon (which potentially needs to be distinct from the Scheduler Horizon)
  * Do we need to simulate an entire observing semester? If so, is there a way of simplifying this? Are we satisfied with just a week, or two weeks?
* Add in random closures of the telescopes, based on weather. See if there is some historical data for how much the telescopes are actually open.

In [15]:
num_scenarios = 3
total_num_requests = 100
bulk_fraction = 0.7
num_injections = 10

def split_requests(num_requests, num_splits):
    # print(num_requests, num_splits)
    split_size = int(num_requests / num_splits)
    remainder = num_requests % num_splits
    print(num_requests, num_splits, split_size, remainder)
    splits = []
    for i in range(num_splits):
        if i < remainder:
            splits.append(split_size + 1)
        else:
            splits.append(split_size)
    return splits

bulk_request_num = round(total_num_requests * bulk_fraction)
print("Bulk Requests:", bulk_request_num)
remaining_request_num = round(total_num_requests * (1 - bulk_fraction))
print("Remaining Requests:", remaining_request_num)
injections = split_requests(remaining_request_num, num_injections)
print("Injections:", injections)
print(sum(injections))

Bulk Requests: 70
Remaining Requests: 30
30 10 3 0
Injections: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
30


In [16]:
for i in range(num_scenarios):
    random.seed(i)
    rg = RequestGeneratorV3(num_telescopes=2)
    # Generate upfront bulk request injection
    rg.generate_single_request_injection(0, bulk_request_num)
    injection_time = 0
    for request_num in split_requests(remaining_request_num, num_injections):
        injection_time += 60*60*3 # Space injections by 3 hours - WHY??? No reason currently.
        rg.generate_single_request_injection(injection_time, request_num)
    filename = "variableWindowsTest_{}.json".format(i)
    rg.save_to_file(filename, "variableWindows_input")

30 10 3 0
Saved to file at 'variableWindows_input\variableWindowsTest_0.json'
30 10 3 0
Saved to file at 'variableWindows_input\variableWindowsTest_1.json'
30 10 3 0
Saved to file at 'variableWindows_input\variableWindowsTest_2.json'


# Testing

In [4]:
obs_data = json.load(open("../archive_scraper/observation_patterns_v2.json", "r"))

In [5]:
sorted_obs_data = sorted(obs_data.values(), key=lambda x: x["cumulative_probability"])

In [10]:
rnum = random.random()

for p in sorted_obs_data:
    if rnum < p["cumulative_probability"]:
        print(p["pattern"])
        print(sum(p["pattern"]))
        break

[1800.0, 60.0, 20.0]
1880.0


In [7]:
sorted_obs_data[0]

{'pattern': [200.0],
 'count': 197,
 'cumulative_probability': 0.08011386742578284}

In [16]:
a = []
for p in sorted_obs_data:
    # print(sum(p["pattern"]))
    a.append(sum(p["pattern"]))

In [13]:
max(a)

10080.0

In [14]:
min(a)

0.0

In [22]:
np.percentile(a, 25)

440.0

In [15]:
24*60*60

86400

In [53]:
def get_pattern():
    rnum = random.random()
    for p in sorted_obs_data:
        if rnum < p["cumulative_probability"]:
        # print(p["pattern"])
        # print(sum(p["pattern"]))
            return sum(p["pattern"])

In [54]:
def generate_req():
    pattern = get_pattern()
    awlength = round(((random.random() * 10) + 1) * pattern)
    return (pattern, awlength)

In [56]:
for i in range(100):
    print(generate_req())

(190.0, 863)
(240.0, 2563)
(3400.0, 36338)
(270.0, 1316)
(2400.0, 24252)
(684.0, 4380)
(480.0, 1531)
(190.0, 1365)
(6490.0, 45771)
(490.0, 3985)
(200.0, 1109)
(1800.0, 3481)
(3700.0, 20982)
(80.0, 813)
(300.0, 430)
(60.0, 522)
(300.0, 664)
(4900.0, 11989)
(480.0, 4531)
(4900.0, 5895)
(420.0, 2423)
(4910.0, 23411)
(1900.0, 13391)
(360.0, 3407)
(120.0, 365)
(4900.0, 28692)
(2240.0, 23228)
(190.0, 1683)
(540.0, 887)
(120.0, 401)
(900.0, 3093)
(600.0, 924)
(300.0, 2271)
(300.0, 812)
(200.0, 256)
(390.0, 408)
(2000.0, 15588)
(200.0, 1223)
(560.0, 1559)
(600.0, 2770)
(3700.0, 7939)
(200.0, 2124)
(900.0, 3146)
(3600.0, 14451)
(80.0, 249)
(155.0, 861)
(240.0, 656)
(560.0, 4104)
(3400.0, 31555)
(600.0, 2997)
(2400.0, 17720)
(560.0, 3844)
(1800.0, 11336)
(3720.0, 38732)
(2000.0, 19223)
(150.0, 1379)
(190.0, 506)
(285.0, 1255)
(6470.0, 64438)
(60.0, 419)
(60.0, 639)
(1400.0, 6453)
(115.0, 543)
(600.0, 5501)
(3700.0, 24435)
(3700.0, 25415)
(2260.0, 14332)
(6470.0, 66148)
(1640.0, 13610)
(1800.0, 3